In [48]:
from tqdm import tqdm
import torch
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

In [42]:
# Get the data and split into train, validation and test sets
X = np.load('data/X.npy')
X = np.moveaxis(X, 1, 2)
y = np.load('data/y.npy')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))
y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test)) 

train_dataloader = DataLoader(TensorDataset(X_train_tensors, y_train_tensors),
                              batch_size=64, shuffle=True, num_workers=8) 
test_dataloader = DataLoader(TensorDataset(X_test_tensors, y_test_tensors),
                             batch_size=64, shuffle=True, num_workers=8) 

(11453, 6, 60) (11453, 3)
(2864, 6, 60) (2864, 3)
----
torch.Size([11453, 6, 60]) torch.Size([11453, 3])
torch.Size([2864, 6, 60]) torch.Size([2864, 3])


In [43]:
class LSTM_test(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM_test, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        return out

In [86]:
num_epochs = 300 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = X_train.shape[2] #number of features
hidden_size = 64 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 3 

model = LSTM_test(num_classes, input_size, hidden_size, num_layers) 

criterion = torch.nn.CrossEntropyLoss()   
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [83]:
def history_logger(history, data):
    for idx, key in enumerate(history):
        history[key].append(data[idx])

In [ ]:
history = {'loss': [], 'val_loss': [], 'acc': [], 'val_acc': []}
for epoch in range(num_epochs):
    loss_total = 0
    val_loss_total = 0
    correct_total = 0
    val_correct_total = 0
    
    model.train()
    for batch_idx, samples in enumerate(train_dataloader):
        outputs = model.forward(samples[0]) #forward pass
        optimizer.zero_grad() #caluclate the gradient, manually setting to 0
        # obtain the loss function
        loss = criterion(outputs, samples[1])
        loss.backward() #calculates the loss of the loss function
        optimizer.step() #improve from loss, i.e backprop
        
        loss_total += loss.item()
        correct_total += (samples[1].argmax(axis=1) == outputs.argmax(axis=1)).float().sum().item()
    
    model.eval()
    with torch.no_grad():
        for batch_idx_test, samples in enumerate(test_dataloader):
            outputs = model.forward(samples[0]) #forward pass
            loss = criterion(outputs, samples[1])

            val_loss_total += loss.item()
            val_correct_total += (samples[1].argmax(axis=1) == outputs.argmax(axis=1)).float().sum().item() 
    
    _loss = loss_total / (batch_idx+1)
    _acc = correct_total / X_train.shape[0]
    _val_loss = val_loss_total / (batch_idx_test+1)
    _val_acc = val_correct_total / X_test.shape[0]
    
    history_logger(history, [_loss, _acc, _val_loss, _val_acc])
    
    print("Epoch: %d, loss: %1.5f, accuracy: %1.5f, val_loss: %1.5f, val_accuracy: %1.5f" % \
          (epoch, _loss, _acc, _val_loss, _val_acc)) 

Epoch: 0, loss: 1.02161, accuracy: 0.50371, val_loss: 1.01133, val_accuracy: 0.50803
Epoch: 1, loss: 1.02004, accuracy: 0.50406, val_loss: 1.01002, val_accuracy: 0.50803
Epoch: 2, loss: 1.01905, accuracy: 0.50406, val_loss: 1.01257, val_accuracy: 0.50803
Epoch: 3, loss: 1.01836, accuracy: 0.50406, val_loss: 1.01345, val_accuracy: 0.50803
Epoch: 4, loss: 1.01480, accuracy: 0.50397, val_loss: 0.99663, val_accuracy: 0.50803
Epoch: 5, loss: 0.98966, accuracy: 0.51716, val_loss: 0.98157, val_accuracy: 0.53142
Epoch: 6, loss: 0.97515, accuracy: 0.52667, val_loss: 0.96756, val_accuracy: 0.52793
Epoch: 7, loss: 0.96881, accuracy: 0.53017, val_loss: 0.96502, val_accuracy: 0.53596
Epoch: 8, loss: 0.96394, accuracy: 0.53174, val_loss: 0.96212, val_accuracy: 0.53736
Epoch: 9, loss: 0.96381, accuracy: 0.53121, val_loss: 0.96498, val_accuracy: 0.53352
Epoch: 10, loss: 0.96493, accuracy: 0.53217, val_loss: 0.97734, val_accuracy: 0.53492
Epoch: 11, loss: 0.96278, accuracy: 0.53672, val_loss: 0.96858, 